In [3]:

from  Kan_Layer import *

import time
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, random_split, Dataset
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torchmetrics.regression import MeanAbsolutePercentageError, R2Score,MeanSquaredError,MeanAbsoluteError
np.random.seed(142)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import time

/Users/ange-clementakazan/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
Abj_dt = pd.read_csv('/Users/ange-clementakazan/Documents/Weather_Prediction_project/Deep-Learning-for-Weather-Prediction/data/Abidjan_data.csv',parse_dates={'datetime': ['YEAR', 'MO','DY']})
# Abj_dt
#Rename Columns
col_name_map = {
    "ALLSKY_SFC_SW_DWN": "ASWD",
    "CLRSKY_SFC_SW_DWN":"CSWD","ALLSKY_SFC_LW_DWN":"ALWD","PRECTOTCORR":"PREC"
}
# new_colnames=['datetime', 'ASWD', 'CSWD',
#        'ALWD', 'T2M', 'T2MDEW', 'T2MWET', 'QV2M', 'RH2M',
#        'PREC', 'PS']
# Rename the columns for both datasets
Abj_dt=Abj_dt.set_index("datetime")
Abj_dt=Abj_dt.rename(columns=col_name_map)
# Xy_train
# target_col = Abj_dt.pop("T2M")
# Abj_dt.insert(len(Abj_dt.columns),"T2M", target_col)
# Abj_dt
target_col = Abj_dt.pop("PREC")
Abj_dt.insert(len(Abj_dt.columns),"PREC", target_col)
Abj_dt

/var/folders/nv/ct8x_h9j78xbj8gnyb35c3_w0000gn/T/ipykernel_40168/1676934423.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  Abj_dt = pd.read_csv('/Users/ange-clementakazan/Documents/Weather_Prediction_project/Deep-Learning-for-Weather-Prediction/data/Abidjan_data.csv',parse_dates={'datetime': ['YEAR', 'MO','DY']})


,ASWD,CSWD,ALWD,T2M,T2MDEW,T2MWET,QV2M,RH2M,PS,PREC
datetime,,,,,,,,,,
2010-01-01,4.06,5.86,412.57,28.05,22.73,25.39,17.33,74.56,100.44,0.08
2010-01-02,5.21,5.58,408.32,28.34,22.48,25.41,17.09,72.56,100.45,0.05
2010-01-03,4.77,5.38,412.82,28.69,22.51,25.60,17.15,71.75,100.41,0.08
2010-01-04,5.17,5.55,407.15,28.44,22.46,25.45,17.15,73.00,100.43,0.05
2010-01-05,5.26,5.57,406.60,28.20,22.46,25.33,17.09,73.50,100.43,0.06
...,...,...,...,...,...,...,...,...,...,...
2023-12-28,4.47,5.38,401.95,27.37,22.82,25.09,17.46,77.75,100.65,0.15
2023-12-29,4.44,5.26,410.70,27.99,24.05,26.01,18.74,80.06,100.55,0.17
2023-12-30,5.12,5.40,409.38,27.72,24.11,25.91,18.80,81.94,100.53,0.26


In [2]:



class TKANCell(nn.Module):
    def __init__(
        self,
        units,
        inputs,
        sub_kan_configs=None,
        sub_kan_output_dim=None,
        sub_kan_input_dim=None,
        activation=torch.tanh,
        recurrent_activation=torch.sigmoid,
        use_bias=True,
        kernel_initializer=nn.init.xavier_uniform_,
        recurrent_initializer=nn.init.orthogonal_,
        bias_initializer=nn.init.zeros_,
        unit_forget_bias=True,
        kernel_regularizer=None,
        recurrent_regularizer=None,
        bias_regularizer=None,
        dropout=0.0,
        recurrent_dropout=0.0,
        seed=None
    ):
        super(TKANCell, self).__init__()
        self.units = units
        self.inputs=inputs
        self.sub_kan_configs = sub_kan_configs or [None]
        self.sub_kan_output_dim = sub_kan_output_dim
        self.sub_kan_input_dim = sub_kan_input_dim
        self.activation = activation
        self.recurrent_activation = recurrent_activation
        self.use_bias = use_bias
        self.dropout = dropout
        self.recurrent_dropout = recurrent_dropout
        self.seed = seed if seed is not None else int(time.time())

        self.kernel = nn.Parameter(torch.Tensor(units * 3, sub_kan_input_dim))
        self.recurrent_kernel = nn.Parameter(torch.Tensor(units * 3, units))

        if self.use_bias:
            self.bias = nn.Parameter(torch.Tensor(units * 3))
        else:
            self.register_parameter("bias", None)

        # Initializing sub-KAN layers
        self.tkan_sub_layers = nn.ModuleList()
        for config in self.sub_kan_configs:
        #     if config is None:
                # layer = KANLinear(self.sub_kan_output_dim, use_layernorm=True)
            layer_method=KANLinear(self.sub_kan_input_dim,self.sub_kan_output_dim)
            layer=layer_method(self.inputs)

            # elif isinstance(config, (int, float)):
            #     layer = KANLinear(self.sub_kan_output_dim, spline_order=config, use_layernorm=True)
            # elif isinstance(config, dict):
            #     layer = KANLinear(self.sub_kan_output_dim, **config, use_layernorm=True)
            # else:
            #     layer = nn.Linear(self.sub_kan_input_dim, self.sub_kan_output_dim)
            self.tkan_sub_layers.append(layer)

        self.sub_tkan_kernel = nn.Parameter(torch.Tensor(len(self.tkan_sub_layers), self.sub_kan_output_dim * 2))
        self.sub_tkan_recurrent_kernel_inputs = nn.Parameter(
            torch.Tensor(len(self.tkan_sub_layers), sub_kan_input_dim, self.sub_kan_input_dim))
        self.sub_tkan_recurrent_kernel_states = nn.Parameter(
            torch.Tensor(len(self.tkan_sub_layers), self.sub_kan_output_dim, self.sub_kan_input_dim))

        self.aggregated_weight = nn.Parameter(torch.Tensor(len(self.tkan_sub_layers) * self.sub_kan_output_dim, self.units))
        self.aggregated_bias = nn.Parameter(torch.Tensor(self.units))

        # Parameter initialization
        self.reset_parameters()

    def reset_parameters(self):
        self.kernel_initializer(self.kernel)
        self.recurrent_initializer(self.recurrent_kernel)
        if self.use_bias:
            if self.unit_forget_bias:
                self.bias.data[:self.units] = 0
                self.bias.data[self.units:2*self.units] = 1
                self.bias.data[2*self.units:] = 0
            else:
                self.bias_initializer(self.bias)
        nn.init.xavier_uniform_(self.sub_tkan_kernel)
        nn.init.xavier_uniform_(self.sub_tkan_recurrent_kernel_inputs)
        nn.init.xavier_uniform_(self.sub_tkan_recurrent_kernel_states)
        nn.init.xavier_uniform_(self.aggregated_weight)
        nn.init.zeros_(self.aggregated_bias)

    def forward(self,  states):
        h_tm1, c_tm1 = states[:2]  # Previous hidden state and cell state
        sub_states = states[2:]  # Sub-layer states
        inputs_=self.inputs
        if self.training and self.dropout > 0.0:
            inputs_ = F.dropout(self.inputs, self.dropout, training=True)
            h_tm1 = F.dropout(h_tm1, self.recurrent_dropout, training=True)

        gates = F.linear(inputs_, self.kernel) + F.linear(h_tm1, self.recurrent_kernel)
        if self.use_bias:
            gates += self.bias

        i, f, c = gates.chunk(3, dim=-1)
        i, f = self.recurrent_activation(i), self.recurrent_activation(f)
        c = f * c_tm1 + i * self.activation(c)

        sub_outputs, new_sub_states = [], []
        for idx, (sub_layer, sub_state) in enumerate(zip(self.tkan_sub_layers, sub_states)):
            agg_input = F.linear(inputs_, self.sub_tkan_recurrent_kernel_inputs[idx]) + \
                        F.linear(sub_state, self.sub_tkan_recurrent_kernel_states[idx])
            sub_output = sub_layer(agg_input)
            sub_recurrent_kernel_h, sub_recurrent_kernel_x = self.sub_tkan_kernel[idx].chunk(2, dim=0)
            new_sub_state = sub_recurrent_kernel_h * sub_output + sub_recurrent_kernel_x * sub_state

            sub_outputs.append(sub_output)
            new_sub_states.append(new_sub_state)

        aggregated_sub_output = torch.cat(sub_outputs, dim=-1)
        aggregated_input = F.linear(aggregated_sub_output, self.aggregated_weight) + self.aggregated_bias
        o = self.recurrent_activation(aggregated_input)
        h = o * self.activation(c)

        return h, [h, c] + new_sub_states


class TKAN(nn.Module):
    def __init__(
        self,
        units,
        inputs,
        **kwargs
    ):
        super(TKAN, self).__init__()
        self.cell = TKANCell(units,inputs, **kwargs)

    def forward(self, sequences, initial_state=None):
        batch_size, timesteps, _ = sequences.shape

        if initial_state is None:
            initial_state = [torch.zeros((batch_size, self.cell.units), device=sequences.device)] * (2 + len(self.cell.tkan_sub_layers))

        outputs, state = [], initial_state
        for t in range(timesteps):
            output, state = self.cell(sequences[:, t, :], state)
            outputs.append(output)

        outputs = torch.stack(outputs, dim=1)
        return outputs, state


In [ ]:
Abj_dt = pd.read_csv('/Users/ange-clementakazan/Documents/Weather_Prediction_project/Deep-Learning-for-Weather-Prediction/data/Abidjan_data.csv',parse_dates={'datetime': ['YEAR', 'MO','DY']})
# Abj_dt
#Rename Columns
col_name_map = {
    "ALLSKY_SFC_SW_DWN": "ASWD",
    "CLRSKY_SFC_SW_DWN":"CSWD","ALLSKY_SFC_LW_DWN":"ALWD","PRECTOTCORR":"PREC"
}
# new_colnames=['datetime', 'ASWD', 'CSWD',
#        'ALWD', 'T2M', 'T2MDEW', 'T2MWET', 'QV2M', 'RH2M',
#        'PREC', 'PS']
# Rename the columns for both datasets
Abj_dt=Abj_dt.set_index("datetime")
Abj_dt=Abj_dt.rename(columns=col_name_map)
# Xy_train
# target_col = Abj_dt.pop("T2M")
# Abj_dt.insert(len(Abj_dt.columns),"T2M", target_col)
# Abj_dt
target_col = Abj_dt.pop("PREC")
Abj_dt.insert(len(Abj_dt.columns),"PREC", target_col)
Abj_dt

In [5]:
# 
scaler = MinMaxScaler(feature_range=(0,1))
def Make_tidydata(data,scaler):
    data= data.sort_index()
    data[data.columns] = scaler.fit_transform(data[data.columns])
    # Splitting the dataset into training, validation, and test sets
    train_size = int(len(data)*0.8)
    val_size =int(train_size*0.10)
    #Train_val_test sets
    Xy_train = data[:train_size-val_size]
    Xy_val= data[train_size-val_size:train_size]
    Xy_test=data[train_size:]
    return Xy_train,Xy_val,Xy_test
Xy_train,Xy_val,Xy_test=Make_tidydata(Abj_dt,scaler)
def dates_splitting(date):
    # Splitting the dataset into training, validation, and test sets
    train_size = int(len(date)*0.8)
    val_size =int(train_size*0.10)
    #Train_val_test sets
    date_train = date[:train_size-val_size]
    date_val= date[train_size-val_size:train_size]
    date_test=date[train_size:]
    return date_train,date_val,date_test

In [6]:
# Define the data 
X_train,y_train=Xy_train.values[:,:-1],Xy_train.values[:,-1]
X_train = torch.tensor(X_train, dtype=torch.float32)  # Convert to float32
y_train = torch.tensor(y_train, dtype=torch.float32)  # Convert to float32

In [7]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# # Generate toy regression data
# X, y = make_regression(n_samples=1000, n_features=5, noise=0.1, random_state=42)

# # Normalize features
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# y = y.reshape(-1, 1)  # Make y a column vector

# # Convert to PyTorch tensors
# X_train = torch.tensor(X, dtype=torch.float32)
# y_train = torch.tensor(y, dtype=torch.float32)

# Create DataLoader
batch_size = 32
train_dataset = TensorDataset(X_train, y_train.reshape(-1, 1))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Training loop
# Define loss and optimizer
criterion = nn.MSELoss()  # Regression loss

epochs = 100
units=1
for epoch in range(epochs):
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        tkan_=TKAN(units, X_batch) 
        optimizer = torch.optim.Adam(tkan_.parameters(), lr=0.01) # This one should not  be inside the for loop, but the network defintion is problemeatic because of the X_batch
        optimizer.zero_grad()  # Reset gradients
         # This 
        # tkan_( X_batch) # The input in this case should be sequences not X_batch
        y_pred = tkan_(X_batch, update_grid=True)  # Forward pass
        loss = criterion(y_pred, y_batch)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        epoch_loss += loss.item()
    
    if epoch % 10 == 0:  # Print loss every 10 epochs
        print(f"Epoch [{epoch}/{epochs}], Loss: {epoch_loss / len(train_loader)}") #:.4f

print("Training complete! 🚀")